In [1]:
data_var = '2024-02-12'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12052,2024-02-12,Israel Superliga,15:10,Hapoel Eilat,Ironi Kiryat Ata,1.97,1.78,164.5,1.86,1.80,1.5,1.81,1.90,tKcQmQHo,0.507614,0.561798,0.537634,0.555556,0.069412,347.522,265.629255,0.764352,0.0,0.000000,NaN,808.50,4.778,3.358611,0.702932,-85.0,280.998,131.457602,0.467824,1.8,1.643168,0.912871,275.12,3.464,1.456376,0.420432,-11.0,77,76,10.50,3.62,155.460,249.034,0.071653,0.023184,0.034307,-2.56,-0.512,-1.894531,0.000000,0.0,0.000000,5.65,1.130,0.690265,0.000000,0.0,0.000000
12053,2024-02-12,Irã Superliga,09:30,Padafan Raad,Mes Kerman,3.98,1.22,156.5,1.83,1.87,-1.5,2.04,1.68,2m7tp0Wu,0.251256,0.819672,0.546448,0.534759,0.070928,473.504,196.374578,0.414726,0.0,0.000000,NaN,720.50,7.606,3.819775,0.502205,-142.0,422.956,476.915977,1.127578,1.8,1.643168,0.912871,191.73,5.434,6.522701,1.200350,1.0,55,83,13.10,2.31,364.904,449.946,0.750621,0.015289,0.136859,-5.00,-1.000,-2.980000,0.000000,0.0,0.000000,5.36,1.072,0.205224,0.000000,0.0,0.000000
12054,2024-02-12,Irã Superliga,09:30,Mahram Tehran,Zob Ahan,1.54,2.37,160.5,1.86,1.84,-4.5,1.92,1.77,2DzzPG0U,0.649351,0.421941,0.537634,0.543478,0.071292,143.104,35.825425,0.250345,1.2,1.643168,1.369306,193.14,1.622,0.462731,0.285284,12.0,128.314,51.277464,0.399625,1.8,1.643168,0.912871,121.44,1.766,0.726347,0.411295,10.0,87,69,2.22,1.76,124.560,126.790,0.300204,0.007644,0.057488,-0.77,-0.154,-3.506494,0.000000,0.0,0.000000,1.38,0.276,4.963768,0.000000,0.0,0.000000
12055,2024-02-12,Irã Superliga,09:30,Averta,Tabiat,2.61,1.45,155.5,1.85,1.85,3.5,2.01,1.70,OMyWPdoO,0.383142,0.689655,0.540541,0.540541,0.072797,209.682,82.631855,0.394082,2.4,1.341641,0.559017,110.67,2.632,1.025753,0.389724,25.0,112.204,19.939191,0.177705,2.4,1.341641,0.559017,104.96,1.404,0.303117,0.215895,56.0,93,82,1.19,1.28,202.308,0.000,0.404061,0.000000,0.118169,1.58,0.316,5.094937,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12056,2024-02-12,Eua Ncaa,22:30,Southern,Alabama State,1.58,2.47,133.5,1.91,1.91,-5.5,1.91,1.80,r7bPw2F8,0.632911,0.404858,0.523560,0.523560,0.037770,109.648,33.270000,0.303426,3.0,0.000000,0.000000,80.04,1.494,0.496719,0.332476,55.0,125.170,32.898172,0.262828,1.2,1.643168,1.369306,142.80,1.920,0.489745,0.255075,-2.0,69,68,1.16,2.10,91.160,128.620,0.310778,0.000000,0.041931,1.31,0.262,2.213740,0.000000,0.0,0.000000,0.03,0.006,245.000000,0.391289,0.5,0.108711
12057,2024-02-12,Europa Wbbl Feminina,14:00,Panevezys F,Liepajas Papirs/LSSS F,5.19,1.14,136.5,1.85,1.85,11.5,1.85,1.85,ptKVEanP,0.192678,0.877193,0.540541,0.540541,0.069871,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,71,68,22.00,1.43,0.000,0.000,0.904829,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12058,2024-02-12,Eua Ncaa,23:00,Towson,Elon,1.09,8.20,136.5,1.80,1.95,-13.5,2.00,1.71,ENGrvOqP,0.917431,0.121951,0.555556,0.512821,0.039382,125.508,19.291705,0.153709,1.8,1.643168,0.912871,98.58,1.892,0.417157,0.220485,6.0,140.848,22.070640,0.156698,1.2,1.643168,1.369306,167.04,2.038,0.394043,0.193348,-17.0,62,64,1.59,2.61,110.744,251.982,1.082353,0.056569,0.110545,1.05,0.210,0.428571,0.608843,0.7,0.091157,-1.37,-0.274,-26.277372,0.320168,0.3,-0.020168
12059,2024-02-12,Eua Nca

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12089,22:00,Eua Nba,Milwaukee Bucks,Denver Nuggets,2.04,Back Home
12103,21:30,Eua Ncaa,Nicholls State,Texas A&M-CC,2.38,Back Home
12104,21:30,Eua Ncaa,Northwestern St.,New Orleans,1.73,Back Home
